### Deploy Model for Inference with SageMaker

Install the sagemaker library

In [2]:
!pip install sagemaker

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 109.6 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=3a19c441229942c5a890d03a81e965372f3403b3b4a1c8847ddefd14ff43fd54
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: attrs
    Found existing installation: attrs 24.2.0
    Uninstalling attrs-24.2.0:
      Successfully uninstalled attrs-24.2.0
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.7

Next, get a Sagemaker session and execution role that will allow you to
create and manage SageMaker resources 

In [3]:
import os
import sagemaker

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"sagemaker role arn: {role}")

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/03/25 23:21:13] INFO     Found credentials from IAM Role: EC2SageMakerIAMRole2              ]8;id=769664;file:///usr/local/lib/python3.10/dist-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=200880;file:///usr/local/lib/python3.10/dist-packages/botocore/credentials.py#1075\1075]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role: EC2SageMakerIAMRole2              ]8;id=978927;file:///usr/local/lib/python3.10/dist-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=986395;file:///usr/local/lib/python3.10/dist-packages/botocore/credentials.py#1075\1075]8;;\

                    INFO     Found credentials from IAM Role: EC2SageMakerIAMRole2              ]8;id=200154;file:///usr/local/lib/python3.10/dist-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=825696;file:///usr/local/lib/python3.10/dist-packages/botocore/credentials.py#1075\1075]8;;\

sagemaker role arn: arn:aws:iam::232087633204:role/EC2SageMakerIAMRole2


We will be using a Deep Learning Container 

In [6]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface-neuronx",
  version="0.0.25"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")


[02/03/25 23:25:55] INFO     Found credentials from IAM Role: EC2SageMakerIAMRole2              ]8;id=669436;file:///usr/local/lib/python3.10/dist-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=408720;file:///usr/local/lib/python3.10/dist-packages/botocore/credentials.py#1075\1075]8;;\

                    INFO     Defaulting to only available Python version: py310                   ]8;id=397188;file:///usr/local/lib/python3.10/dist-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=556561;file:///usr/local/lib/python3.10/dist-packages/sagemaker/image_uris.py#608\608]8;;\

                    INFO     Defaulting to only supported image scope: inf2.                      ]8;id=326494;file:///usr/local/lib/python3.10/dist-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=753031;file:///usr/local/lib/python3.10/dist-packages/sagemaker/image_uris.py#532\532]8;;\

llm image uri: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.25-neuronx-py310-ubuntu22.04


Deploy the compiled model to a SageMaker real-time endpoint on AWS
Inferentia.  

Change user_id below to your Hugging Face username. Make sure to update
HF_MODEL_ID and HUGGING_FACE_HUB_TOKEN with your Hugging Face username
and your access token.

In [ ]:
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.inf2.24xlarge"
health_check_timeout=2400 # additional time to load the model
volume_size=512 # size in GB of the EBS volume

# Define Model and Endpoint configuration parameter
config = {
    "HF_MODEL_ID": "liorsadan/Mixtral-8x7B-Instruct-v0.1.25", # replace with your model id if you are using your own model
    "HF_NUM_CORES": "4", # number of neuron cores
    "HF_AUTO_CAST_TYPE": "fp16",  # dtype of the model
    "MAX_BATCH_SIZE": "1", # max batch size for the model
    "MAX_INPUT_LENGTH": "1000", # max length of input text
    "MAX_TOTAL_TOKENS": "1024", # max length of generated text
    "MESSAGES_API_ENABLED": "true", # Enable the messages API
    "HUGGING_FACE_HUB_TOKEN": "YOUR_HUGGINGFACE_TOKEN" # Add your Hugging Face token here
}


# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  env=config
)

[02/03/25 23:29:17] INFO     Found credentials from IAM Role: EC2SageMakerIAMRole2              ]8;id=659289;file:///usr/local/lib/python3.10/dist-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=561320;file:///usr/local/lib/python3.10/dist-packages/botocore/credentials.py#1075\1075]8;;\

You are now ready to deploy the model to a SageMaker real-time inference
endpoint.  SageMaker will provision the necessary compute resources
instance, retrieve and launches the inference container, which downloads
the model artifacts from your Hugging Face repository, loads the model
to the Inferentia devices and starts inference serving.  This process
can take several minutes.

In [9]:
# Deploy model to an endpoint
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm_model._is_compiled_model = True # We precompiled the model

llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout,
  volume_size=volume_size
)

[02/03/25 23:29:20] INFO     Creating model with name:                                              ]8;id=757830;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py\session.py]8;;\:]8;id=701741;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-02-03-23-29-20-943                       

[02/03/25 23:29:21] INFO     Creating endpoint-config with name                                     ]8;id=319460;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py\session.py]8;;\:]8;id=602926;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py#5889\5889]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-02-03-23-29-21-549                       

                    INFO     Creating endpoint with name                                            ]8;id=389455;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py\session.py]8;;\:]8;id=320626;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py#4711\4711]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-02-03-23-29-21-549                       

-------------------------------!

Next, run a simple test to check the endpoint. First, update user_id to
match your Hugging Face username, then create the prompt and parameters.

and send the prompt to the SageMaker real-time endpoint for inference

In [25]:
# Prompt to generate
messages=[
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "Create a program in python that: 1. Asks for the user's name, 2. Handles empty input, and 3. Returns a greeting" }
]

# Generation arguments
parameters = {
    "model": "liorsadan/Mixtral-8x7B-Instruct-v0.1.25", # replace user_id
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 800,
}

and send the prompt to the SageMaker real-time endpoint for inference

In [30]:
chat = llm.predict({"messages" :messages, **parameters})

print(chat["choices"][0]["message"]["content"].strip())

Here is a simple Python program that meets your requirements:

```python
# Ask for the user's name
name = input("Please enter your name: ")

# Handle empty input
while name == "":
    print("Sorry, you can't leave the name empty.")
    name = input("Please enter your name: ")

# Return a greeting
print("Hello, " + name + "! Nice to meet you.")
```

In this program, the user is asked to enter their name. If the user enters an empty string, the program will display a message asking them to enter a name again. This will continue until the user enters a name that is not an empty string. Once a valid name is entered, the program will display a greeting.</s>


In [28]:
# Ask for the user's name
name = input("Please enter your name: ")

# Handle empty input
while name == "":
    print("Sorry, you can't leave your name empty. Please try again.")
    name = input("Please enter your name: ")

# Return a greeting
print("Hello, " + name + "! Nice to meet you.")

Please enter your name:  


Sorry, you can't leave your name empty. Please try again.


Please enter your name:  Yair


Hello, Yair! Nice to meet you.


In the future if you want to connect to this inference endpoint from
other applications, first find the name of the inference endpoint:

In [12]:
endpoints = sess.sagemaker_client.list_endpoints()

for endpoint in endpoints['Endpoints']:
    print(endpoint['EndpointName'])

huggingface-pytorch-tgi-inference-ml-in-2025-02-03-23-29-21-549


Alternatively, use the AWS console \> SageMaker \> Inference \>
Endpoints to see a list of all the SageMaker endpoints deployed in your
account.

Use this endpoint name to update the following code, which can be ran in
other locations.

In [29]:
from sagemaker.huggingface import HuggingFacePredictor

endpoint_name="huggingface-pytorch-tgi-inference-ml-in-2025-02-03-23-29-21-549"

llm = HuggingFacePredictor(
  endpoint_name=endpoint_name,
  sagemaker_session=sess
)

### Cleanup

Make sure to delete the endpoint to stop paying for the provisioned
resources


In [31]:
llm.delete_model()
llm.delete_endpoint()

[02/04/25 00:01:13] INFO     Deleting model with name:                                              ]8;id=97936;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py\session.py]8;;\:]8;id=368939;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py#5226\5226]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-02-03-23-29-20-943                       

                    INFO     Deleting endpoint configuration with name:                             ]8;id=382361;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py\session.py]8;;\:]8;id=718084;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py#4865\4865]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-02-03-23-29-21-549                       

                    INFO     Deleting endpoint with name:                                           ]8;id=270054;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py\session.py]8;;\:]8;id=285296;file:///usr/local/lib/python3.10/dist-packages/sagemaker/session.py#4855\4855]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-02-03-23-29-21-549                       